# Edge Probing Predictions Sandbox

Use this notebook as a starting point for #datascience on Edge Probing predictions. The code below (from `probing/analysis.py`) will load predictions from a run, do some pre-processing for convenience, and expose two DataFrames for analysis.

We load the data into Pandas so it's easier to filter by various fields, and to select particular columns of interest (such as `labels.khot` and `preds.proba` for computing metrics). For an introduction to Pandas, see here: https://pandas.pydata.org/pandas-docs/stable/10min.html 

In [1]:
import sys, os, re, json
import itertools
import collections
from importlib import reload
import pandas as pd
import numpy as np
from sklearn import metrics

The latest runs are here:

In [2]:
ls /nfs/jsalt/home/iftenney/exp/edges-20180727/

elmo-chars-edges-constituent-ptb/  elmo-full-edges-constituent-ptb/
elmo-chars-edges-coref-ontonotes/  elmo-full-edges-coref-ontonotes/
elmo-chars-edges-dep-labeling/     elmo-full-edges-dep-labeling/
elmo-chars-edges-dpr/              elmo-full-edges-dpr/
elmo-chars-edges-ner-conll2003/    elmo-full-edges-ner-conll2003/
elmo-chars-edges-spr2/             elmo-full-edges-spr2/
elmo-chars-edges-srl-conll2005/    elmo-full-edges-srl-conll2005/


The `elmo-chars` experiments probe the char CNN layer only (lexical baseline), while the `elmo-full` models use full ELMo with learned mixing weights. The run dir for each is just called "run" by default. 

In [3]:
import analysis
reload(analysis)

run_dir = "/nfs/jsalt/home/iftenney/exp/edges-20180727/elmo-full-edges-spr2/run"
preds = analysis.Predictions.from_run(run_dir, 'edges-spr2', 'test')
print("Number of examples: %d" % len(preds.example_df))
print("Number of total targets: %d" % len(preds.target_df))
print("Labels (%d total):" % len(preds.all_labels))
print(preds.all_labels)

Loading vocabulary from /nfs/jsalt/home/iftenney/exp/edges-20180727/elmo-full-edges-spr2/vocab
Loading predictions from /nfs/jsalt/home/iftenney/exp/edges-20180727/elmo-full-edges-spr2/run/edges-spr2_test.json
Number of examples: 276
Number of total targets: 582
Labels (20 total):
['awareness', 'change_of_location', 'change_of_possession', 'change_of_state', 'change_of_state_continuous', 'changes_possession', 'existed_after', 'existed_before', 'existed_during', 'exists_as_physical', 'instigation', 'location_of_event', 'makes_physical_contact', 'partitive', 'predicate_changed_argument', 'sentient', 'stationary', 'volition', 'was_for_benefit', 'was_used']


### Top-level example info

`preds.example_df` contains information on the top-level examples. Mostly, this just stores the input text and any metadata fields that were present in the original data. This is useful if you want to link the targets back to the text, but you shouldn't need it to compute most metrics.

In [4]:
preds.example_df.head()

,idx,info.grammatical,info.sent-id,info.sent_id,info.source,info.split,preds.proba,text
idx,,,,,,,,
0,0,5.0,1008,1008,SPR2,test,"[[0.9611670970916748, 0.06295457482337952, 0.0...","In a timid voice , he says : &quot; If an airp..."
1,1,5.0,1009,1009,SPR2,test,"[[0.838303804397583, 0.13125662505626678, 0.01...",&quot; Wonderful ! &quot; Winston beams .
2,2,5.0,1017,1017,SPR2,test,"[[0.9852041006088257, 0.015768878161907196, 0....",&quot; Our new lunar transportation system uti...
3,3,2.0,1023,1023,SPR2,test,"[[0.9831392168998718, 0.0872965157032013, 0.01...",They want to use LTS to tie into NASA &apos; s...
4,4,5.0,1024,1024,SPR2,test,"[[0.9918457865715027, 0.02819274738430977, 0.0...",&quot; We are so excited that the White House ...


### Target info and predictions

`preds.target_df` contains the per-target input fields (`span1`, `span2`, and `label`) as well as any metadata associated with individual targets. The `idx` column references a row in `example_df` that this target belongs to, if you need to recover the original text.

The loader code does some preprocessing for convenience. In particular, we add a `label.ids` column which maps the list-of-string `label` column into a list of integer ids for these targets, as well as `label.khot` which contains a K-hot encoding of these ids. 

Each entry in `label.khot` should align to the corresponding entry in `preds.proba`, which contains the model's predicted probabilities $\hat{y} \in [0,1]$ for each class.

For specific analysis, it might be easier to work with the wide and long forms of this DataFrame - see cells below.

In [5]:
preds.target_df.head()

,idx,info.is_pilot,info.pred_lemma,info.span1_text,info.span2_txt,label,preds.proba,span1,span2,label.ids,label.khot
0,0,False,say,says,he,"[awareness, existed_after, existed_before, exi...","[0.9611670970916748, 0.06295457482337952, 0.00...","[6, 7]","[5, 6]","[0, 6, 7, 8, 10, 15, 17, 19]","[1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, ..."
1,0,False,carry,carrying,winston peters,"[awareness, change_of_location, change_of_stat...","[0.7967376112937927, 0.9281209707260132, 0.254...","[12, 13]","[13, 15]","[0, 1, 4, 6, 7, 8, 10, 15, 17, 18, 19]","[1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, ..."
2,0,False,blow,blown,an airplane carrying winston peters,"[change_of_location, change_of_state, existed_...","[0.22298480570316315, 0.7640542387962341, 0.11...","[16, 17]","[10, 15]","[1, 3, 7, 8]","[0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, ..."
3,1,False,beam,beams,winston,"[awareness, change_of_state_continuous, existe...","[0.838303804397583, 0.13125662505626678, 0.016...","[5, 6]","[4, 5]","[0, 4, 6, 7, 8, 10, 13, 15, 17, 18, 19]","[1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, ..."
4,2,False,tell,told,kistler,"[awareness, existed_after, existed_before, exi...","[0.9852041006088257, 0.015768878161907196, 0.0...","[30, 31]","[29, 30]","[0, 6, 7, 8, 10, 15, 17, 19]","[1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, ..."


### Wide and Long Data

For background on these views, see https://altair-viz.github.io/user_guide/data.html#long-form-vs-wide-form-data

Here's a "wide" version of the data, with the usual metadata plus `2* num_labels` columns: `label.true.<label_name>` and `preds.proba.<label_name>` for each target class.

In [6]:
preds.target_df_wide.head()

Generating wide-form target DataFrame. May be slow...


,idx,info.is_pilot,info.pred_lemma,info.span1_text,info.span2_txt,span1,span2,label.true.awareness,label.true.change_of_location,label.true.change_of_possession,...,preds.proba.instigation,preds.proba.location_of_event,preds.proba.makes_physical_contact,preds.proba.partitive,preds.proba.predicate_changed_argument,preds.proba.sentient,preds.proba.stationary,preds.proba.volition,preds.proba.was_for_benefit,preds.proba.was_used
0,0,False,say,says,he,"[6, 7]","[5, 6]",1,0,0,...,0.939851,0.002335,0.002191,0.302883,0.001822,0.980764,0.001951,0.949263,0.132487,0.955613
1,0,False,carry,carrying,winston peters,"[12, 13]","[13, 15]",1,1,0,...,0.686126,0.002442,0.004153,0.098275,0.003371,0.638721,0.001872,0.524297,0.278615,0.940805
2,0,False,blow,blown,an airplane carrying winston peters,"[16, 17]","[10, 15]",0,1,0,...,0.193996,0.002598,0.005462,0.204345,0.002103,0.176916,0.001414,0.026365,0.115171,0.841224
3,1,False,beam,beams,winston,"[5, 6]","[4, 5]",1,0,0,...,0.926281,0.002403,0.003535,0.161786,0.002510,0.897601,0.001807,0.796774,0.450231,0.962066
4,2,False,tell,told,kistler,"[30, 31]","[29, 30]",1,0,0,...,0.917733,0.004447,0.003747,0.294328,0.002832,0.943016,0.003232,0.979802,0.636887,0.969834


We can fairly easily compute per-label metrics from the wide form, by selecting the appropriate pair of columns:

In [7]:
wide_df = preds.target_df_wide
scores_by_label = {}
for label in preds.all_labels:
    y_true = wide_df['label.true.' + label]
    y_pred = wide_df['preds.proba.' + label] >= 0.5
    score = metrics.f1_score(y_true=y_true, y_pred=y_pred)
    scores_by_label[label] = score
scores = pd.Series(scores_by_label)
print(scores)
print("Macro average F1: %.04f" % scores.mean())

awareness                     0.900641
change_of_location            0.274194
change_of_possession          0.048780
change_of_state               0.323529
change_of_state_continuous    0.553106
changes_possession            0.000000
existed_after                 0.951949
existed_before                0.920354
existed_during                0.987826
exists_as_physical            0.000000
instigation                   0.811124
location_of_event             0.000000
makes_physical_contact        0.000000
partitive                     0.055556
predicate_changed_argument    0.000000
sentient                      0.900840
stationary                    0.000000
volition                      0.865350
was_for_benefit               0.599138
was_used                      0.923077
dtype: float64
Macro average F1: 0.4558


/usr/share/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


And here's a "long" version of the same, with a single `label` column, and one column each for `label.true` and `preds.proba` for that label:

In [8]:
preds.target_df_long.head()

Generating long-form target DataFrame. May be slow...


,idx,info.is_pilot,info.pred_lemma,info.span1_text,info.span2_txt,label,label.true,preds.proba,span1,span2
0,0,False,say,says,he,awareness,1,0.961167,"[6, 7]","[5, 6]"
1,0,False,say,says,he,partitive,0,0.302883,"[6, 7]","[5, 6]"
2,0,False,carry,carrying,winston peters,partitive,0,0.098275,"[12, 13]","[13, 15]"
3,0,False,blow,blown,an airplane carrying winston peters,partitive,0,0.204345,"[16, 17]","[10, 15]"
4,0,False,blow,blown,an airplane carrying winston peters,was_used,0,0.841224,"[16, 17]","[10, 15]"


We can easily get the set of labels available here:

In [9]:
preds.target_df_long.label.unique()

array(['awareness', 'partitive', 'was_used', 'predicate_changed_argument',
       'change_of_possession', 'sentient', 'stationary',
       'change_of_location', 'volition', 'was_for_benefit',
       'existed_after', 'change_of_state', 'existed_before',
       'existed_during', 'changes_possession', 'exists_as_physical',
       'instigation', 'change_of_state_continuous', 'location_of_event',
       'makes_physical_contact'], dtype=object)

And easily compute micro-averaged metrics by simply comparing the `label.true` and `preds.proba` columns:

In [10]:
from sklearn import metrics
long_df = preds.target_df_long
metrics.f1_score(y_true=long_df['label.true'], y_pred=(long_df['preds.proba'] >= 0.5))

0.8334710256107636